In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import kraft

In [ ]:
import numpy as np

In [ ]:
ve1 = np.array([np.nan, 0, 1, 2])

ve2 = np.array([4, 8, 0, np.nan])

In [ ]:
def plot_x_y(x_, y_, xaxis_title, yaxis_title, title=None):

    kraft.plot.plot_plotly(
        {
            "layout": {
                "title": title,
                "xaxis": {
                    "title": xaxis_title,
                },
                "yaxis": {
                    "title": yaxis_title,
                },
            },
            "data": [
                {
                    "x": x_,
                    "y": y_,
                }
            ],
        }
    )

## get_entropy

In [ ]:
kraft.information.get_entropy(ve1)

In [ ]:
x_ = []

y_ = []

for po in range(10):

    n_da = 2 ** po

    x_.append(n_da)

    y_.append(kraft.information.get_entropy(np.full(n_da, 1)))

plot_x_y(x_, y_, "N", "Entropy")

## get_kld

In [ ]:
kraft.information.get_kld(ve1, ve2)

In [ ]:
x_ = []

y_ = []

for po in range(10):

    n_da = 2 ** po

    x_.append(n_da)

    y_.append(
        kraft.information.get_kld(
            np.random.random_sample(n_da), np.random.random_sample(n_da)
        ).sum()
    )

plot_x_y(x_, y_, "N", "KLD")

## get_jsd

In [ ]:
kraft.information.get_jsd(ve1, ve2)

In [ ]:
x_ = []

y_ = []

for po in range(10):

    n_da = 2 ** po

    x_.append(n_da)

    y_.append(
        kraft.information.get_jsd(
            np.random.random_sample(n_da), np.random.random_sample(n_da)
        )[-1].sum()
    )

plot_x_y(x_, y_, "N", "JSD")

## get_zd

In [ ]:
kraft.information.get_zd(ve1, ve2)

In [ ]:
x_ = []

y_ = []

for po in range(10):

    n_da = 2 ** po

    x_.append(n_da)

    y_.append(
        kraft.information.get_zd(
            np.random.random_sample(n_da), np.random.random_sample(n_da)
        )[-1].sum()
    )

plot_x_y(x_, y_, "N", "ZD")

## get_ic

In [ ]:
try:

    kraft.information.get_ic(ve1, ve2)

except ValueError as er:

    print(er)

In [ ]:
for ve1, ve2 in [
    [[0] * 8] * 2,
    [[1] * 8] * 2,
    [np.arange(8), np.arange(8)[::-1]],
    [[0, 1, 2, 3, 4, 5, 6, 7, 8, 7, 6, 5, 4, 3, 2, 1, 0]] * 2,
    [[8, 7, 6, 5, 4, 3, 2, 1, 0, 1, 2, 3, 4, 5, 6, 7, 8]] * 2,
    [[0, 1] * 8, [0, 1] * 8],
    [[0, 1] * 8, [1, 0] * 8],
]:

    print()

    ve1 = np.array(ve1)

    ve2 = np.array(ve2)

    print(ve1)

    print(ve2)

    print(kraft.information.get_ic(ve1, ve2))

In [ ]:
ve1 = np.array([0, 0, 1, 1, 2, 2])

for fa in [1, 2, 1 / 2, 10, 1 / 10]:

    print()

    ve2 = ve1 * fa

    print(ve2)

    print(kraft.information.get_ic(ve1, ve2))

In [ ]:
x_ = []

y_ = []

for po in range(10):

    n = 2 ** po

    x_.append(n)

    y_.append(
        kraft.information.get_ic(
            np.random.random_sample(n_da), np.random.random_sample(n_da)
        )
    )

plot_x_y(x_, y_, "N", "IC")

## get_icd

In [ ]:
n_da = 1000

kraft.information.get_icd(np.random.random_sample(n_da), np.random.random_sample(n_da))